# `Python Webscraping Indeed`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ `Webscrape Indeed`: take position, job title,location,date of posting, string of qualifications

+ Use a list of words related to your skills or skills you are interested in and extract from job post qualifications section.

+ Date time formating

`------------------------------`

# `Next video Word Cloud and NLP using these data`


In [5]:
# pip3 install wordcloud #(if you don't have already)
import pandas as pd
import numpy as np
import PIL              # for images
import wordcloud        # ploting word cloud
import requests         # grab web-page
import pickle
from bs4 import BeautifulSoup as bsopa  # parse web-page
import datetime

In [ ]:
# if you don't have wordcloud, get that now:

# !pip3 install wordcloud
# conda install -c conda-forge wordcloud

In [181]:

"""Inside this container I have:
    company name, job tile, job skills needed, city, state"""

data = []
matches=[]
def indeed_scrape(soupie,position,location):
    for i in range(0,100,10):
        y=requests.get('https://www.indeed.com/jobs?q={}&l={}&sort=date='.format(position, location)+str(i))
        
#         y=requests.get('https://www.indeed.com/jobs?q=data+scientist&l=california&sort=date='+str(i))
        sou=bsopa(y.text,'lxml')
    
        for i in sou.find_all('div', {'class': 'row'}):
#         for i in sou.find_all('div',{"class":"jobsearch-SerpJobCard"}):
            
            job_title = i.find('a', {'data-tn-element': 'jobTitle'})['title']
            company_name = i.find('span', {'class': 'company'}).text.strip()    
            location=i.find('span',{"class":"location"})
            post_date = i.find('span', attrs={'class': 'date'})
            if location:
                location=location.text.strip()
            else:
                location=i.find('div',{"class":"location"})
                location=location.text.strip()           
    
            summary_position=i.find(href=True)
            summary_position=summary_position['href']
            summary_position=str(summary_position).replace('&amp;','&')

            response2=requests.get('https://www.indeed.com'+summary_position)
            job_summary_file=bsopa(response2.text,'lxml')

            if job_summary_file.find('div',{'class':'jobsearch-jobDescriptionText'}):
                matches.append(job_summary_file.find('div',
                {'class':'jobsearch-jobDescriptionText'}).text.strip())
            else:
                matches.append('')

#-----------
            for i in matches:
                datum = {'job_title': job_title,
                    'company_name': company_name,
                    'location': location,
                    'post_Date':post_date,
                    'Qualifications': i}

            data.append(datum)
    return data


# indeed_scrape(soupie)

In [253]:
jobs_=pd.DataFrame(indeed_scrape(sou,'data scientist','california'))

In [263]:
print(len(jobs_))
jobs_.head()

453


,job_title,company_name,location,post_Date,Qualifications,skill_matches
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",09-08-2020,,[]
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",09-24-2020,Data ScientistLaxmi Therapeutic Devices – Gole...,[algorithms]
2,Data Scientist,Blue Owl,"San Francisco, CA",09-08-2020,"Our Mission\nWe’re here to create a safer, hap...","[deep learning, pandas]"
3,Data Scientist,Triplebyte,California,09-19-2020,About Triplebyte\n\nTriplebyte is transforming...,"[aws, machine learning]"
4,"Data Scientist, Analytics",Evernote,"Redwood City, CA 94063 (Downtown area)",10-07-2020,About the team:\n\nThe Evernote Analytics team...,"[statistics, visualization]"


# Let'd do a few things here:

+ Create a DF
+ dump the file as a text file and store as a pkl 

In [256]:
jobs_.to_csv('Indeed_scrape_2020CA.txt')

# `Create a list of skills that you may have or general list`

+ This can be used in a few ways, for example if I put all my skills and see what matched up for jobs.

+ We could generate a corpus and start parsing through job postings not just data scientist, but similar jobs as well. After all the matches for indeed are not the best!

In [257]:
buzz_words=['Python','SQL','AWS', 'Machine learning','Deep learning','Text mining',
'NLP','SAS','Tableau','Sagemaker','Tensorflow','Spark', 'numpy', 'MongDB','PSQL',
"Postgres", 'Pandas', 'RESTFUL','NLP','Statistics','Algorithms','Visualization',
'GCP','Google Cloud','Naive Bayes','Random Forest','Bachelors degree','Masters degree']
buzz_words_list=[x.lower() for x in buzz_words]  # convert list to lowercase to parse


In [270]:

yo=[]
for i in range(len(jobs_.Qualifications)):
    a=buzz_words_list
    dd=[x for x in a if x in jobs_.Qualifications[i].lower()]
    yo.append(dd)
jobs_['skill_matches']=yo
jobs_.head(10)

,job_title,company_name,location,post_Date,Qualifications,skill_matches
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",09-08-2020,,[]
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",09-24-2020,Data ScientistLaxmi Therapeutic Devices – Gole...,"[python, sql, statistics, algorithms]"
2,Data Scientist,Blue Owl,"San Francisco, CA",09-08-2020,"Our Mission\nWe’re here to create a safer, hap...","[python, machine learning, deep learning, pandas]"
3,Data Scientist,Triplebyte,California,09-19-2020,About Triplebyte\n\nTriplebyte is transforming...,"[aws, machine learning]"
4,"Data Scientist, Analytics",Evernote,"Redwood City, CA 94063 (Downtown area)",10-07-2020,About the team:\n\nThe Evernote Analytics team...,"[python, sql, sas, tableau, statistics, visual..."
5,Data Scientist,Airspace Technologies Inc,"Carlsbad, CA 92010",10-07-2020,"Who Are We?\nAt Airspace Technologies, we live...","[python, aws, machine learning, deep learning,..."
6,Data Scientist,"RMDS Lab, Inc","South Lake, CA",09-23-2020,About RMDS LabRMDS Lab and its Global Associat...,"[python, machine learning]"
7,Data Scientist,Airspace Technologies,"Carlsbad, CA 92010",10-07-2020,Job Details\nJob Location\nAirspace Technologi...,"[python, aws, machine learning, deep learning,..."
8,Data Scientist,ELT Solutions,"San Jose, CA",10-05-2020,"Location: San Jose, CA, Irving, TX\nSalary: DO...","[sql, spark, statistics]"
9,Data Scientist (Full Time; Multiple Openings),RingCentral,"Belmont, CA",10-02-2020,Data Scientist (Full Time; Multiple Openings)\...,"[python, sql, machine learning, deep learning,..."


# `Format dates:`

In [260]:
v=[]
for i in jobs_['post_Date']:

    if re.findall(r'[0-9]',i.text):
        # if the string has digits convert each entry to single string: ['3','0']->'30'
        b=''.join(re.findall(r'[0-9]',i.text))
        
        # convert string int to int and subtract from today's date and format
        g=(datetime.datetime.today()-datetime.timedelta(int(b))).strftime('%m-%d-%Y')

        v.append(g)
        
    else: # this will contain strings like: 'just posted' or 'today' etc before convert
        v.append(datetime.datetime.today().strftime('%m-%d-%Y'))
v[:5]

['09-08-2020', '09-24-2020', '09-08-2020', '09-19-2020', '10-07-2020']

In [271]:
# updated posting dates! yay--
jobs_['post_Date']=v
jobs_.head()

,job_title,company_name,location,post_Date,Qualifications,skill_matches
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",09-08-2020,,[]
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",09-24-2020,Data ScientistLaxmi Therapeutic Devices – Gole...,"[python, sql, statistics, algorithms]"
2,Data Scientist,Blue Owl,"San Francisco, CA",09-08-2020,"Our Mission\nWe’re here to create a safer, hap...","[python, machine learning, deep learning, pandas]"
3,Data Scientist,Triplebyte,California,09-19-2020,About Triplebyte\n\nTriplebyte is transforming...,"[aws, machine learning]"
4,"Data Scientist, Analytics",Evernote,"Redwood City, CA 94063 (Downtown area)",10-07-2020,About the team:\n\nThe Evernote Analytics team...,"[python, sql, sas, tableau, statistics, visual..."


# <font color=red>LIKE</font>, Share &

# <font color=red>SUB</font>scribe

# `Next Video: Word Cloud From Indeed Job Postings, with NLP`

`--------------------`

https://www.jobspikr.com/blog/scraping-indeed-job-data-using-python/

https://jlgamez.com/how-i-scrape-jobs-data-from-indeed-com-with-python/

http://rstudio-pubs-static.s3.amazonaws.com/495949_1ab68b7e55cf4a3ab1fc00285e28fbcc.html

https://chrislovejoy.me/job-scraper/

https://www.youtube.com/watch?v=eN_3d4JrL_w